
Import the necessary modules

In [411]:
import random as rd
import numpy as np
import utils
import time
import sys

from heapq import heappop, heappush
from utils import pretty_print_timetable
from check_constraints import check_mandatory_constraints

In [412]:
class Day:
    def __init__(self, name):
        self.name = name

In [413]:
class Interval:
    def __init__(self, interval):
        self.interval = interval

In [414]:
class Subject:
    def __init__(self, name, capacity):
        self.name = name
        self.capacity = capacity
        self.occupied = 0
    
    def is_covered(self):
        return self.occupied >= self.capacity

    def update(self, lecture_hall):
        self.occupied += lecture_hall.capacity

    def restart(self):
        self.occupied = 0

In [415]:
class Constraints:
    def __init__(self, constraints):
        days = ["Luni", "Marti", "Miercuri", "Joi", "Vineri", "Sambata", "Duminica"]
        self.c_days = []
        self.c_intervals = []
        self.c_pauses = []
        
        for c in constraints:
            if c in days or ''.join(list(c)[1:]) in days:
                self.c_days.append(c)
            elif "-" in c:
                self.c_intervals.append(c)
            else:
                self.c_pauses.append(c)

In [416]:
class Teacher:
    def __init__(self, name, constraints, subjects):
        self.name = name
        self.num_intervals = 0
        self.constraints = Constraints(constraints)
        self.subjects = subjects

    def is_available(self):
        return self.num_intervals < 7

    def is_specialized(self, s):
        """
            Tells if the teacher may teach the subject s received as a parameter
        """
        return s in self.subjects

    def update(self):
        self.num_intervals += 1

    def restart(self):
        self.num_intervals = 0

In [417]:
class LectureHall:
    def __init__(self, name, capacity, subjects):
        self.name = name
        self.capacity = capacity
        self.subjects = subjects

In [432]:
class Slot:
    def __init__(self, day, interval, lecture_hall):
        self.day = day
        self.interval = interval
        self.lecture_hall = lecture_hall

        self.teacher = None
        self.subject = None

    def is_available(self):
        return (self.subject is None) or (self.teacher is None)

    def matches_requirements(self, lecture_hall, teacher, slots):
        return self.lecture_hall.name == lecture_hall.name and\
            not any(slot.teacher == teacher for slot in [s for s in slots if s.interval == self.interval and s.day == self.day])

    def update(self, subject, teacher):
        self.subject = subject
        self.teacher = teacher

    def restart(self):
        self.subject = None
        self.teacher = None

In [419]:
class Timetable:
    def __init__(self, days, interval, lecture_halls):
        self.slots = []
        for d in days:
            for i in interval:
                for l in lecture_halls:
                    self.slots.append(Slot(d, i, l))

In [420]:
def process_data(data):
    # Initialize Days and Intervals
    days = [Day(day_name) for day_name in data["Zile"]]
    intervals = [Interval(interval) for interval in data["Intervale"]]

    # Initialize subjects
    subjects = [Subject(name, capacity) for name, capacity in data["Materii"].items()]
    
    # Initialize lecture halls
    lecture_halls = [LectureHall(name, details["Capacitate"], [s for s in subjects if s.name in details["Materii"]]) for name, details in data["Sali"].items()]

    # Create empty timetable
    timetable = Timetable(days, intervals, lecture_halls)

    # Initialize teachers
    teachers = [Teacher(name, details["Constrangeri"], [s for s in subjects if s.name in details["Materii"]]) for name, details in data["Profesori"].items()]
    
    return {
        "timetable": timetable,
        "days": days,
        "intervals": intervals,
        "subjects": subjects,
        "lecture_halls": lecture_halls,
        "teachers": teachers
    }

In [421]:
def print_data(data):
    print("Timetable Slots:")
    for slot in data["timetable"].slots:
        slot_details = f"{slot.day.name} {slot.interval.interval}, Subject: {slot.subject.name if slot.subject else 'None'}, "
        slot_details += f"Teacher: {slot.teacher.name if slot.teacher else 'None'}, "
        slot_details += f"Hall: {slot.lecture_hall.name if slot.lecture_hall else 'None'}"
        print(slot_details)

    print("\nSubjects:")
    for subject in data["subjects"]:
        print(f"{subject.name}, Capacity: {subject.capacity}, Occupied: {subject.occupied}")

    print("\nLecture Halls:")
    for hall in data["lecture_halls"]:
        subjects = ', '.join([s.name for s in hall.subjects])
        print(f"{hall.name}, Capacity: {hall.capacity}, Subjects: {subjects}")

    print("\nTeachers:")
    for teacher in data["teachers"]:
        subjects = ', '.join([s.name for s in teacher.subjects])
        constraints = f"Days: {', '.join(teacher.constraints.c_days)}, Intervals: {', '.join(teacher.constraints.c_intervals)}, Pauses: {', '.join(teacher.constraints.c_pauses)}"
        print(f"{teacher.name}, Subjects: {subjects}, Constraints: {constraints}")

In [422]:
def get_interval_tuple(interval):
    splitted = interval[1:len(interval) - 1].split(",")
    first = int(splitted[0])
    second = int(splitted[1][1:])
    return (first, second)

In [423]:
def save_timetable(data, file_path):
    processed_data = {}

    for d in data["days"]:
        processed_data[d.name] = {}
        for i in data["intervals"]:
            processed_data[d.name][get_interval_tuple(i.interval)] = {}
            
    for slot in data["timetable"].slots:
        processed_data[slot.day.name][get_interval_tuple(slot.interval.interval)][slot.lecture_hall.name] = (slot.teacher.name, slot.subject.name) if slot.teacher else None
    
    pretty_data = pretty_print_timetable(processed_data, file_path)

    with open(f"outputs/{file_path.split('/')[1].split('.')[0]}.txt", "w") as f:
        f.writelines(pretty_data)
    

Function to run the algorithm


In [424]:
def run_algorithm(file_path, func, h=None) -> None:
    data = process_data(utils.read_yaml_file(file_path))

    if h is not None:
        data["h"] = h

    # TODO run the algorithm
    func(data, file_path)

In [425]:
def is_solution(timetable, specs):
    return not check_mandatory_constraints(timetable.state, specs)

In [426]:
def astar(data):
    h = data["h"]
    
    # Initialize empty timetable
    start_timetable = Timetable(0, h(data), h, data)
    
    OPEN = []
    CLOSED = set()
    heappush(OPEN, (start_timetable.f, start_timetable))
    
    while OPEN:
        _, curr = heappop(OPEN)

        if is_solution(curr, data):
            return curr

        CLOSED.add(curr)
        for SUCC in curr.generate_successors():
            if SUCC not in CLOSED:
                heappush(OPEN, (SUCC.f, SUCC))
    
    
    

In [427]:
def h1():
    pass

In [428]:
class RandomRestartHillClimbing():
    def _all_subjects_covered(subjects):
        return all(s.is_covered() for s in subjects)
    
    def generate_initial_state_restart(data, restarts):
        for slot in data["timetable"].slots:
            slot.restart()

        for s in data["subjects"]:
            s.restart()
    
        for t in data["teachers"]:
            t.restart()

        restarts += 1

        sys.stdout.write(f"\rRestarts: {restarts}")
        sys.stdout.flush()

        return data, restarts
    
    def generate_initial_state(data):
        restarts = 0
        print("Starting to generate the initial state...")

        while not RandomRestartHillClimbing._all_subjects_covered(data["subjects"]):
            # Choose a random subject
            s = rd.choice([s for s in data["subjects"] if not s.is_covered()])

            while not s.is_covered():
                # Choose a random lecture hall
                try:
                    l = rd.choice([l for l in data["lecture_halls"] if s in l.subjects])
                except IndexError:
                    # if there is an IndexError, it means that the choice() method was applied on an empty list
                    # therefore there are no lecture halls available so the generating algorithm has to be restarted
                    data, restarts = RandomRestartHillClimbing.generate_initial_state_restart(data, restarts)
                    break

                # Choose a random teacher
                try:
                    t = rd.choice([t for t in data["teachers"] if t.is_available() and t.is_specialized(s)])
                except IndexError:
                    # if there is an IndexError, it means that the choice() method was applied on an empty list
                    # therefore there are no teachers available so the generating algorithm has to be restarted
                    data, restarts = RandomRestartHillClimbing.generate_initial_state_restart(data, restarts)
                    break

                # Choose a random slot
                try:
                    slot = rd.choice([slot for slot in data["timetable"].slots\
                        if slot.is_available() and slot.matches_requirements(l, t, data["timetable"].slots)])
                except IndexError:
                    # if there is an IndexError, it means that the choice() method was applied on an empty list
                    # therefore there are no slots available so the generating algorithm has to be restarted
                    data, restarts = RandomRestartHillClimbing.generate_initial_state_restart(data, restarts)
                    break
            
                slot.update(s, t)
                s.update(l)
                t.update()
        # sys.stdout.write("\n")
        # sys.stdout.flush()
        print("Generating the initial state - DONE")
        return data


In [429]:
def random_restart_hill_climbing(data, file_path):
    # Generate the initial state
    S = RandomRestartHillClimbing.generate_initial_state(data)

    save_timetable(S, file_path)
    

Run the algorithm for "dummy.yaml"

In [434]:
run_algorithm("inputs/dummy.yaml", random_restart_hill_climbing)

Starting to generate the initial state...
Generating the initial state - DONE


Run the algorithm for "orar_constrans_incalcat.yaml"

In [ ]:
run_algorithm("inputs/orar_constrans_incalcat.yaml", h=h1)

TypeError: run_algorithm() missing 1 required positional argument: 'func'

Run the algorithm for "orar_mic_exact.yaml"

In [ ]:
run_algorithm("inputs/orar_mic_exact.yaml", h=h1)

Run the algorithm for "orar_mediu_relaxat.yaml"

In [ ]:
run_algorithm("inputs/orar_mediu_relaxat.yaml", h=h1)

Run the algorithm for "orar_mare_relaxat.yaml"

In [ ]:
run_algorithm("inputs/orar_mare_relaxat.yaml", h=h1)

Run the algorithm for all files

In [435]:
input_files = [
    "inputs/dummy.yaml",
    "inputs/orar_bonus_exact.yaml",
    "inputs/orar_constrans_incalcat.yaml",
    "inputs/orar_mare_relaxat.yaml",
    "inputs/orar_mediu_relaxat.yaml",
    "inputs/orar_mic_exact.yaml",
]

for file in input_files:
    run_algorithm(file, random_restart_hill_climbing)

Starting to generate the initial state...
Generating the initial state - DONE
Starting to generate the initial state...
Restarts: 4Generating the initial state - DONE
Starting to generate the initial state...
Generating the initial state - DONE
Starting to generate the initial state...
Generating the initial state - DONE
Starting to generate the initial state...
Generating the initial state - DONE
Starting to generate the initial state...
Generating the initial state - DONE
